# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "C:\\Users\\bpiet\\anaconda3\\envs\\PythonData\\Class\\Weather_Database\\WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
print(city_data_df.head())

           City Country      Lat      Lng  Max Temp  Humidity  Cloudiness  \
0  Punta Arenas      CL -53.1500 -70.9167     46.51        61          40   
1         Turda      RO  46.5667  23.7833     39.04       100          82   
2  Staryy Nadym      RU  65.6118  72.6842    -12.89        99          49   
3       Kananga      CD  -5.8958  22.4178     67.46        99         100   
4        Shache      CN  38.4167  77.2406     44.87        26          76   

   Wind Speed Current Description  
0       34.52    scattered clouds  
1        1.14                 fog  
2        4.61    scattered clouds  
3        1.48          light rain  
4        0.27       broken clouds  


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [1]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("Minimum acceptable temperature: "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("Maximum acceptable temperature: "))

Minimum acceptable temperature: 10
Maximum acceptable temperature: 45


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [ ]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & (city_data_df['Max Temp']<=max_temp)]
                                    
# Display sample data
print(preferred_cities_df.head())

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [ ]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
print(clean_travel_cities.head())

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']>=min_temp) & (city_data_df['Max Temp']<=max_temp)]

# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


# Display sample data
print(hotel_df.head())

NameError: name 'city_data_df' is not defined

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [ ]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
print(hotel_df.head())

## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [ ]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [46]:
## Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = hotel_df['Lat'][index]
    lng = hotel_df['Lng'][index]

    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Minimum acceptable temperature: 10
Maximum acceptable temperature: 15
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 49.6901546, 'lng': -83.66677109999999}, 'viewport': {'northeast': {'lat': 49.69130278029149, 'lng': -83.66509431970849}, 'southwest': {'lat': 49.68860481970849, 'lng': -83.66779228029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Companion', 'opening_hours': {'open_now': True}, 'photos': [{'height': 398, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113281167746351545527">Companion</a>'], 'photo_reference': 'AW30NDyOoxajLMYRUWqZT7ObGHBdhjdnu7GYqwSjaIJ1zEhWyWvAiREAQKo2Ynw3elN2fuLrs1Hm7yMn83KqnkOllGDMcWGp8Sk0Ija68dIVO5DM-O8NDlQQ2qli6paLwcCIWPJU6yGqLLfB3pIby23Nk3FvYomVhlnivchjXE4jbDyw4-dv', 'width': 995}], 'place_

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 56.0803153, 'lng': 56.5388682}, 'viewport': {'northeast': {'lat': 56.08169103029149, 'lng': 56.5406574802915}, 'southwest': {'lat': 56.0789930697085, 'lng': 56.5379595197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Uyut', 'place_id': 'ChIJNZNbwn-p3UMRJH6hupjx2Ds', 'reference': 'ChIJNZNbwn-p3UMRJH6hupjx2Ds', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'vicinity': '3GJQ+4G, Askino'}], 'status': 'OK'}
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 59.03497969999999, 'lng': 80.8779359}, 'viewport': {'northeast': {'lat': 59.0363781302915, 'lng': 80.8792118302915}, 'southwest': {'lat': 59.03368016970848, 'lng': 80

{'html_attributions': [], 'next_page_token': 'AW30NDxlxJ1Dq4AxE2Tl4BO4-Be3n8z2O1firKF8upuisHADZ8pdcSSYxd5Wi_wBbYCN0iSoNluKdpvTIU_qSzLCPmm25mNoaFoY8igdbVN6CvDSez0xwBSnSjvcKRQtbJ6VHIYTiasOhr3O_V7JIl7b7a5Z_eCHcrDJgKbZeGsbCaAg6UiBEsKKeeLxokm2EveCs92ChbZVHl28EqOdUvweG9rQ_OAJPCbRZgPRe0L2V6oP_sHYBDUfG2_MVcoqm_CQ5wg058xiwLlXpmKjpvD9_Ij827uD7EOmtUhnj4KreD_vctIsZBVSDhfFOxdMoYoLN3IGuPu97VkQW_wpEM_ZbjZQlucPiR-0INFoFnMhevXJLLecfZ2OmpnhE46MxA1dNbaFrnq9yHGedfB-v5YfMOiTcyLLnJG4i4BPzbyi27LghaZGykrHPtsd', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 34.5636366, 'lng': 76.1225418}, 'viewport': {'northeast': {'lat': 34.5649366802915, 'lng': 76.12396203029151}, 'southwest': {'lat': 34.5622387197085, 'lng': 76.12126406970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Caravan Sarai Kargil

In [47]:
# Display sample data
print(hotel_df.head())

         City Country  Max Temp Current Description      Lat      Lng  \
8      Hearst      CA     12.45     overcast clouds  49.6835 -83.6665   
14   Sladkovo      RU     12.60     overcast clouds  55.5284  70.3385   
272  Sisimiut      GL     14.58     overcast clouds  66.9395 -53.6735   
472    Askino      RU     12.85     overcast clouds  56.0901  56.5783   
497  Kargasok      RU     14.32          light snow  59.0556  80.8572   

         Hotel Name  
8         Companion  
14       Gostinitsa  
272  Hotel Sisimiut  
472            Uyut  
497  Hotel Kargasok  


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [51]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
print(clean_hotel_df.head(10))

            City Country  Max Temp Current Description      Lat      Lng  \
8         Hearst      CA     12.45     overcast clouds  49.6835 -83.6665   
14      Sladkovo      RU     12.60     overcast clouds  55.5284  70.3385   
272     Sisimiut      GL     14.58     overcast clouds  66.9395 -53.6735   
472       Askino      RU     12.85     overcast clouds  56.0901  56.5783   
497     Kargasok      RU     14.32          light snow  59.0556  80.8572   
559  Krasnoborsk      RU     14.95     overcast clouds  61.5598  45.9340   
638       Kargil      IN     14.56     overcast clouds  34.5667  76.1000   

                        Hotel Name  
8                        Companion  
14                      Gostinitsa  
272                 Hotel Sisimiut  
472                           Uyut  
497                 Hotel Kargasok  
559  Baza Otdykha "Medvezhiy Ugol"  
638     Hotel Caravan Sarai Kargil  


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [50]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("C:\\Users\\bpiet\\anaconda3\\envs\\PythonData\\Class\\Vacation_Search\\WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [52]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
output_data_file = clean_hotel_df.to_csv(output_data_file, index=False)

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [54]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

SyntaxError: invalid syntax (4204937732.py, line 2)

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [55]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Companion</dd>\n<dt>City</dt><dd>Hearst</dd>\n<dt>Country</dt><dd>CA</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 12.45 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Gostinitsa</dd>\n<dt>City</dt><dd>Sladkovo</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 12.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Sisimiut</dd>\n<dt>City</dt><dd>Sisimiut</dd>\n<dt>Country</dt><dd>GL</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 14.58 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Uyut</dd>\n<dt>City</dt><dd>Askino</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 12.85 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Kargasok</dd>\n<dt>City</dt><dd>Kargasok</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>light snow and 14.32 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Baza Otdykha "Medvezhiy Ugol"</dd>\n<dt>City</dt><dd>Kra

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [ ]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [57]:
#Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Display the figure containing the map
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))